**IMPORT LIBRERIAS Y DATA**

In [1]:
# Acceder a la base de datos
import sqlite3
# Convertir a dataframe data
import pandas as pd
# Matrices
import numpy as np

# Filtrado colaborativo
from surprise import SVD, Reader, Dataset
# Filtrado por contenido
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Paths hacia la base de datos de google
google_db = r'../data/google/production/google.db'
yelp_db = r'../data/yelp/production/yelp.db'

<hr>

**CONEXION A BASE DE DATOS**

In [3]:
# Realizamos la conexion con la base de datos de google y yelp
google_conn = sqlite3.connect(google_db)
yelp_conn = sqlite3.connect(yelp_db)

# Creamos los correspondientes cursors.
google_cursor = google_conn.cursor()
yelp_cursor = yelp_conn.cursor()

In [4]:
# Obtenemos las categorias de google metadata
google_cursor.execute('SELECT DISTINCT agg_categories FROM metadata')
google_categories = [x[0] for x in google_cursor.fetchall()]

# Obtenemos las categorias de yelp business
yelp_cursor.execute('SELECT DISTINCT agg_categories FROM business')
yelp_categories = [x[0] for x in yelp_cursor.fetchall()]

<hr>

**FUNCIONES**

*PARTE 1*

In [5]:
def get_visited_ids(user_id, cursor, use_google=True):
    # Retorna el id de aquellas empresas que el usuario ha visitado.
    if use_google:
        cursor.execute('SELECT company_index FROM user_company WHERE user_index = ?', (user_id,))
        retrieve = eval(cursor.fetchall()[0][0])
    else:
        cursor.execute('SELECT company_index FROM reviews WHERE user_index = ?', (user_id,))
        retrieve = [x[0] for x in cursor.fetchall()]
    return retrieve

*PARTE 2*

In [6]:
def compute_distance(user_coord, company_coord):
    # Computa la distancia entre user_coord y company_coord en KM usando latitude y longitude
    lat1, lon1, lat2, lon2 = [np.radians(x) for x in [user_coord[0], user_coord[1], company_coord[0], company_coord[1]]]
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    temp = (  
        np.sin(dlat / 2) ** 2 
        + np.cos(lat1) 
        * np.cos(lat2) 
        * np.sin(dlon / 2) ** 2
    )
    distance = 6373.0 * (2 * np.arctan2(np.sqrt(temp), np.sqrt(1 - temp)))
    return distance

def filter_companies(user_coord, category, visited_ids, threshold=10, cursor=google_cursor, use_google=True):
    # Funcion que retorna una lista con las empresas y su metadata segun la filtracion.
    
    # Convertimos a string y eliminamos los corchetes
    visited_ids = str(visited_ids)[1:-1]
    # Creamos las querys correspondientes
    google_query = f'SELECT company_index, latitude, longitude, state FROM metadata WHERE company_index NOT IN ({visited_ids}) AND state IS NOT NULL AND agg_categories = "{category}";'
    yelp_query = f'SELECT company_index, latitude, longitude FROM business WHERE company_index NOT IN ({visited_ids}) AND agg_categories = "{category}";'

    # Si queremos un sistema de recomendacion con google
    if use_google:
        cursor.execute(google_query)
        retrieve = cursor.fetchall()
    # Si queremos un sistema de recomendacion con yelp
    else:
        cursor.execute(yelp_query)
        retrieve = cursor.fetchall()
    
    # Lista que contendra company_index, state, latitude, longitude, distance from user de aquellos con distance <= threshold
    output = []
    for row in retrieve:
        company_coord = row[1], row[2]
        distance = compute_distance(user_coord, company_coord)
        if distance <= threshold:
            row = list(row) + [distance]
            output.append(row)

    return output

*PARTE 3*

In [7]:
def state_companies(data):
    # Funcion que retorna un diccionario que mapea estado - lista de empresas. Esta diseña para google.
    output = {}
    for row in data:
        estado = row[3]
        company_id = row[0]
        if estado not in output:
            output[estado] = [company_id]
        else:
            output[estado].append(company_id)
    return output

def get_reviews(data, cursor=google_cursor, use_google=True):
    # Funcion que retorna user_index, company_index, rating/stars de aquellas empresas en data.
    if use_google:
        # El caso de google requiere un manejo especial puesto que las reviews estan por estados en diferentes tablas.
        retrieve = []
        for state, companies in state_companies(data).items():
            companies = str(companies)[1:-1]
            cursor.execute(f'SELECT user_index, company_index, rating FROM {state} WHERE company_index in ({companies});')
            retrieve.extend(cursor.fetchall())
    else:
        companies = str([x[0] for x in data])[1:-1]
        cursor.execute(f'SELECT user_index, company_index, stars FROM reviews WHERE company_index IN ({companies});')
        retrieve = cursor.fetchall()
    return retrieve

*PARTE 4*

In [8]:
def make_predictions(user_id, reviews, rating_threshold=4.):
    # Funcion que retorna el id de aquellas empresas en reviews que el modelo predice que el usuario mejor calificaria.
    
    # Creamos el lector
    reader = Reader(rating_scale=(1, 5))
    # Creamos el dataframe temporal
    data = pd.DataFrame(reviews, columns=['user_index', 'company_index', 'rating'])
    # Almacenamos los ids de las empresas
    no_visited_ids = list(data['company_index'].unique())
    # Convertimos dataframe a dataset
    data = Dataset.load_from_df(data[['user_index', 'company_index', 'rating']], reader)
    # Creamos el training set
    trainset = data.build_full_trainset()
    # Creamos el modelo
    svd_model = SVD()
    # Lo entrenamos
    svd_model.fit(trainset)
    # Realizamos y retornamos las predicciones
    return [id for id in no_visited_ids if svd_model.predict(user_id, id).est >= rating_threshold]

*PARTE 5*

In [9]:
def get_recommendations(visited_ids, predict_ids, use_google=True, similarity_threshold=0.2):
    # Funcion que devuelve el ids de las empresas recomendadas
    
    # Paths hacia la matrix de vectores
    google_vectors = r'../data/google/production/company_vectors.npy'
    yelp_vectors = r'../data/yelp/production/company_vectors.npy'
    # Datatype de los elementos de los vectores
    data_type = np.float64
    
    if use_google:
        # Matrix shape para los vectores de google
        matrix_shape = (2997736, 150)
        memmap_array = np.memmap(google_vectors, dtype=data_type, mode='r', shape=matrix_shape)
    else:
        # Matrix shape para los vectores de yelp
        matrix_shape = (150309, 150)
        memmap_array = np.memmap(yelp_vectors, dtype=data_type, mode='r', shape=matrix_shape)

    # Obtenemos los vectores de las empresas no visitadas
    no_visited_vectors = memmap_array[predict_ids, :]
    # Obtenemos los vectores de las empresas ya visitadas
    visited_vectors = memmap_array[visited_ids, :]
    # Agrupamos en uno, los vectores de las empresas ya visitadas
    agg_visited_vectors = np.mean(visited_vectors, axis=0)
    # Calculamos la similitud de coseno entre agg_visited y no_visited vectors
    sim_matrix = cosine_similarity([agg_visited_vectors], no_visited_vectors).flatten()
    # Retornamos los id de las recomendaciones cuya similitud sea mayor al threshold definido
    return [ predict_ids[index] for index in np.argsort(sim_matrix).flatten()[::-1][1:] if sim_matrix[index] >= similarity_threshold]

*PARTE 6*

In [10]:
def get_companies(ids, cursor, use_google=True):
    # Ultima funcion que obtiene los ids de las recomendaciones y retorna informacion sobre dichas empresas.

    # Convertimos la lista de id a string para poder realizar la query
    ids = str(ids)[1:-1]
    # Columnas para obtener los datos de sus respectivas bases de datos
    google_columns = ['company_index', 'name', 'address', 'latitude', 'longitude', 'category', 'avg_rating', 'num_of_reviews', 'hours', 'misc']
    yelp_columns = ['company_index', 'name', 'address', 'latitude', 'longitude', 'categories', 'stars', 'review_count', 'hours', 'attributes']
    
    if use_google:
        cursor.execute(f'SELECT {",".join(google_columns)} FROM metadata WHERE company_index IN ({ids});')
       
    else:
        cursor.execute(f'SELECT {",".join(yelp_columns)} FROM business WHERE company_index IN ({ids});')
    # Pase lo que pase, le mandamos la data con el mismo nombre de columnas. En este caso decidimos usar la de yelp por ser nombres mas intuitivos.
    return pd.DataFrame(cursor.fetchall(), columns=yelp_columns).to_json()

*PARTE 7*

In [11]:
def main(user_id, user_coord, category, use_google, threshold):
    # Funcion que ejecuta las partes 1 al 6 y retorna las recomendaciones.
    
    # Paths hacia las bases de datos
    google_db = r'../data/google/production/google.db'
    yelp_db = r'../data/yelp/production/yelp.db'
    
    # Realizamos la conexion con la base de datos de google y yelp
    # Y, creamos el cursor correspondiente.
    
    if use_google:
        google_conn = sqlite3.connect(google_db)
        cursor = google_conn.cursor()
    else:
        yelp_conn = sqlite3.connect(yelp_db)
        cursor = yelp_conn.cursor()
        
    # Parte 1
    visited_ids = get_visited_ids(user_id, cursor, use_google)
    # Parte 2
    filter_company_data = filter_companies(user_coord, category, visited_ids, threshold=threshold, cursor=cursor, use_google=use_google)
    # Parte 3
    reviews = get_reviews(filter_company_data, cursor=cursor, use_google=use_google) 
    # Parte 4
    predict_ids = make_predictions(user_id, reviews)
    # Parte 5
    recommendations_ids = get_recommendations(visited_ids, predict_ids, use_google=use_google)
    # Parte 6
    recommendations_data = get_companies(recommendations_ids, cursor, use_google=use_google)
    # Parte 7
    return recommendations_data

<hr>

**USER INPUT**

In [12]:
user_id = 1
user_coord = [36.1433929232, -86.8141224616]
threshold = 100 # km
use_google = True
cursor = google_cursor if use_google else yelp_cursor
category = 'retail stores'

In [13]:
data = main(user_id, user_coord, category, use_google, threshold=threshold)